In [ ]:
import base
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

### Setup

- Load data
- Add metadata
- Draw gates
- Gate transfected cells

Result from this section: DataFrame `df` representing transfected cells.

Load all transfection data related to circuit tuning (miR, num target sites, promoter)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'

exp90_path = base_path/'2024.03.31_exp90'/'export'
exp90_2_path = base_path/'2024.04.02_exp90.2'/'export'
exp90_3_path = base_path/'2024.04.02_exp90.3'/'export'
exp90_4_path = base_path/'2024.04.05_exp90.4'/'export'
exp91_path = base_path/'2024.04.08_exp91'/'export'
exp92_path = base_path/'2024.04.12_exp92'/'export'

plates = pd.DataFrame({
    'data_path': [exp90_path/'plate1', exp90_path/'plate2', 
                  exp90_2_path, exp90_4_path,
                  exp90_3_path/'plate1', exp90_3_path/'plate2', 
                  exp91_path/'plate1.1', exp91_path/'plate1.2', exp91_path/'plate1.3', 
                  exp91_path/'plate2.1', exp91_path/'plate2.2', exp91_path/'plate2.3',
                  exp92_path/'plate1.1', exp92_path/'plate1.2', exp92_path/'plate1.3', 
                  exp92_path/'plate2.1', exp92_path/'plate2.2', exp92_path/'plate2.3',],
    
    'yaml_path': ([exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', 
                   exp90_path/'exp90_plate2_wells.yaml', exp90_path/'exp90_plate1_wells.yaml',
                   exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', ] +
                  [exp91_path/'exp91_plate1_wells.yaml']*3 + 
                  [exp91_path/'exp91_plate2.1_wells.yaml', exp91_path/'exp91_plate2.2_wells.yaml', exp91_path/'exp91_plate2.3_wells.yaml'] +
                  [exp92_path/'exp92_plate1_wells.yaml', exp92_path/'exp92_plate1.2_wells.yaml', exp92_path/'exp92_plate1_wells.yaml',
                   exp92_path/'exp92_plate2_wells.yaml', exp92_path/'exp92_plate2.2_wells.yaml', exp92_path/'exp92_plate2_wells.yaml',]
                  ),
    
    'biorep': ([1, 1, 
                2, 2, 
                3, 3,] + 
                [1, 2, 3,]*4),
    
    'exp': (['exp90', 'exp90', 
             'exp90.2', 'exp90.4', 
             'exp90.3', 'exp90.3',] + 
            ['exp91']*6 + 
            ['exp92']*6)
})

output_path = rd.rootdir/'output'/'tuning'
cache_path = rd.rootdir/'output'/'tuning'/'data.gzip'

metadata_keys = set()
for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)
    metadata_keys.update(rd.flow.load_well_metadata(p).keys())
display(metadata_keys)

In [ ]:
# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color']
construct_markers = metadata_dict['markers']

Draw gates based on untransfected population, then gate transfected cells

In [ ]:
gates = pd.DataFrame()
channel_list = ['tagBFP-A', 'mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing gates
gates.loc[len(gates.index)] = ['exp90.4',0,0,0,0,]  
gates.loc[gates['exp']=='exp90.4', channel_list] = gates.loc[gates['exp']=='exp90.2', channel_list].values

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

display(gates)

Gate data per experiment based on transfection marker expression

In [ ]:
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
max = 1e6
df = data[(data['expressing']) & (data['output']<max)]
display(df)

### Explore miRE-FF4 data

In [ ]:
mir = 'miRE-FF4-hPGK'
output_path = output_path/mir
mir_df = df[(df['promoter']=='hPGK') & 
            ((df['group']=='base') | # no miR + no TS
             ((df['group'].isin(['ts3', 'ts5'])) & (df['ts'].isin(['FF6x1', 'FF4x1']))) | # TS only
             ((df['group']=='miR') & (df['miR']=='miRE.FF4')) | # miR only, controller
             ((df['group']=='controller') & (df['miR']=='miRE.FF4') & (df['ts'].isin(['FF6x1', 'FF4x1'])))
            )].copy()
mir_df.sort_values('design', inplace=True)
group_order = {'base': 1, 'miR': 2, 'ts3': 3, 'ts5': 4, 'controller': 5}
mir_df.sort_values('group', key=lambda x: x.map(group_order), inplace=True)
mir_df.sort_values('ts_kind', inplace=True)
display(mir_df)

group_order = {'base': 1, 'miR': 2, 'ts3': 3, 'ts5': 4, 'controller': 5}

##### Calculate statistics on bulk & binned data

In [ ]:
mir_df['marker'] = mir_df['marker'].astype(float)
mir_df['output'] = mir_df['output'].astype(float)

# Group and compute stats
stat_list = [np.mean, np.std, sp.stats.gmean, sp.stats.gstd, sp.stats.variation]
grouped = mir_df.groupby(by=['construct','biorep','exp'])
stats = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats.columns = stats.columns.map(lambda i: base.rename_multilevel_cols(i))
stats['count'] = grouped['marker'].count().reset_index()['marker']
stats = stats.merge(metadata, how='left', on='construct')
stats.sort_values(['design','ts_kind','ts_num'], inplace=True)

display(stats)

metadata['TS'] = metadata['ts_kind']
stat_name = {'output_gmean': 'Geometric\nmean', 'output_std': 'Standard\ndeviation', 'output_variation': 'Coefficient\nof variation'}

In [ ]:
# Bin by transfection marker
min_count = 100
mir_df['bin_marker'] = mir_df.groupby(['construct','exp'])['marker'].transform(lambda x: pd.cut(x, np.logspace(2,6,15)))
mir_df['remove_bin'] = mir_df.groupby(['construct','exp','bin_marker'])['bin_marker'].transform(lambda x: x.count() < min_count)
df_binned = mir_df[~mir_df['remove_bin']].copy()
df_binned['marker'] = df_binned['marker'].astype(float)
df_binned['output'] = df_binned['output'].astype(float)

# Group and compute stats
stat_list = [np.mean, np.std, sp.stats.gmean, sp.stats.gstd, sp.stats.variation]
grouped = df_binned.groupby(by=['construct','biorep','exp','bin_marker'])
stats_bin = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_bin.columns = stats_bin.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_bin['count'] = grouped['marker'].count().reset_index()['marker']
stats_bin = stats_bin.merge(metadata, how='left', on='construct')

# Compute mean/median on bin span
df_binned['bin_marker_mean'] = df_binned['bin_marker'].map(lambda x: np.mean([x.left, x.right]))
df_binned['bin_marker_median'] = df_binned['bin_marker'].map(lambda x: np.median([x.left, x.right]))
stats_bin['bin_marker_mean'] = stats_bin['bin_marker'].map(lambda x: np.mean([x.left, x.right]))
stats_bin['bin_marker_median'] = stats_bin['bin_marker'].map(lambda x: np.median([x.left, x.right]))

display(stats_bin)

In [ ]:
# Combine bioreps
mir_df['remove_bin_combined'] = mir_df.groupby(['construct','bin_marker'])['bin_marker'].transform(lambda x: x.count() < min_count)
df_binned_combined = mir_df[~mir_df['remove_bin_combined']].copy()

stat_list = [np.mean, np.std, sp.stats.gmean, sp.stats.gstd, sp.stats.variation]
grouped = df_binned_combined.groupby(by=['construct','bin_marker'])
stats_bin_combined = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

stats_bin_combined.columns = stats_bin_combined.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_bin_combined['count'] = grouped['marker'].count().reset_index()['marker']
stats_bin_combined = stats_bin_combined.merge(metadata, how='left', on='construct')

df_binned_combined['bin_marker_mean'] = df_binned_combined['bin_marker'].map(lambda x: np.mean([x.left, x.right]))
df_binned_combined['bin_marker_median'] = df_binned_combined['bin_marker'].map(lambda x: np.median([x.left, x.right]))
stats_bin_combined['bin_marker_mean'] = stats_bin_combined['bin_marker'].map(lambda x: np.mean([x.left, x.right]))
stats_bin_combined['bin_marker_median'] = stats_bin_combined['bin_marker'].map(lambda x: np.median([x.left, x.right]))

In [ ]:
# Normalize output to gmean of output in smallest bin, and normalize marker bin by smallest bin
def normalize_output(df):
    df = df.copy()
    normalizer = sp.stats.gmean(df.loc[(df['bin_marker_median']==df['bin_marker_median'].min()), 'output'])
    df['output_norm'] = df['output'].astype(float) / normalizer
    df['bin_marker_median_norm'] = df['bin_marker_median'].astype(float) / df['bin_marker_median'].min()
    return df

by = ['construct','biorep','exp']
df_binned = df_binned.groupby(by)[df_binned.columns].apply(normalize_output).reset_index(drop=True)
display(df_binned)

# Cache data
df_binned.loc[:, ~df_binned.columns.isin(['color','bin_marker'])].to_parquet(rd.outfile(output_path/('df_binned_'+mir+'.gzip')))

##### Begin by looking just at controls

In [ ]:
# Remove targeting controller
mir_controls_df = mir_df[~((mir_df['group']=='controller') & (mir_df['ts_kind']=='T'))].copy()

# Create color palette where NT controller is colored by design (instead of gray)
metadata['color_controls'] = metadata['color']
metadata.loc[(metadata['group']=='controller') & (metadata['design']==1), 'color_controls'] = base.colors['teal']
metadata.loc[(metadata['group']=='controller') & (metadata['design']==2), 'color_controls'] = base.colors['orange']
metadata.loc[(metadata['group']=='controller') & (metadata['design']==3), 'color_controls'] = base.colors['red']
metadata_dict = metadata.set_index('construct').to_dict('dict')
controls_palette = metadata_dict['color_controls']

In [ ]:
plot_df = mir_controls_df
plot_df.sort_values('group', key=lambda x: x.map(group_order), inplace=True)
g = sns.displot(data=plot_df[plot_df['group']!='base'], x='output', hue='construct', 
                palette=controls_palette, row='biorep', col='group', kind='kde', log_scale=True, legend=False, 
                common_norm=False, facet_kws=dict(margin_titles=True, sharex=True, sharey=True),)
for (biorep, group), ax in g.axes_dict.items():
    sns.kdeplot(data=plot_df[(plot_df['group']=='base') & (plot_df['biorep']==biorep)], x='output', 
                log_scale=True, legend=False, common_norm=False, ax=ax, color='black', ls=':')
g.figure.savefig(rd.outfile(output_path/(f'hist_'+mir+'_controls.svg')), bbox_inches='tight')

In [ ]:
plot_df = df_binned[~((df_binned['group']=='controller') & (df_binned['ts_kind']=='T'))]
plot_df.sort_values('group', key=lambda x: x.map(group_order), inplace=True)
g = sns.relplot(data=plot_df, row='biorep', col='group', facet_kws=dict(sharex=True, sharey=True, margin_titles=True,), kind='line',
                    height=3, aspect=1.3, x='bin_marker_median', y='output', hue='construct',
                    palette=controls_palette, legend=False, dashes=False, style='construct', markers=construct_markers,
                    estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
g.set(xscale='log', yscale='log',)
sns.despine()
for name, ax in g.axes_dict.items(): ax.grid()
g.figure.savefig(rd.outfile(output_path/(f'line_'+mir+'_controls.svg')), bbox_inches='tight')

In [ ]:
# Normalize output to gmean of output in smallest bin, and normalize marker bin by smallest bin
def normalize_output_to_base(df):
    df = df.copy()
    normalizer = sp.stats.gmean(df.loc[(df['group']=='base'), 'output'])
    df['output_norm_to_base'] = df['output'].astype(float) / normalizer
    return df

by = ['bin_marker_median','biorep','exp']
df_norm_base = df_binned.groupby(by)[df_binned.columns].apply(normalize_output_to_base).reset_index(drop=True).dropna()
display(df_norm_base)

In [ ]:
# Group and compute stats
stat_list = [np.mean, np.std, sp.stats.gmean, sp.stats.gstd, sp.stats.variation]
grouped = df_norm_base.groupby(by=['construct','biorep','exp'])
stats_norm_base = grouped[['output_norm_to_base']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_norm_base.columns = stats_norm_base.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_norm_base['count'] = grouped['marker'].count().reset_index()['marker']
stats_norm_base = stats_norm_base.merge(metadata, how='left', on='construct')
stats_norm_base.sort_values(['design','ts_kind','ts_num'], inplace=True)

display(stats_norm_base)

In [ ]:
plot_df = df_norm_base[(df_norm_base['group']!='base') & ~((df_norm_base['group']=='controller') & (df_norm_base['ts_kind']=='T'))]
plot_df.sort_values('group', key=lambda x: x.map(group_order), inplace=True)
g = sns.relplot(data=plot_df, row='biorep', col='group', facet_kws=dict(sharex=True, sharey=True, margin_titles=True,), kind='line',
                    height=3, aspect=1.3, x='bin_marker_median', y='output_norm_to_base', hue='construct',
                    palette=controls_palette, legend=False, dashes=False, style='construct', markers=construct_markers,
                    estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
g.set(xscale='log', yscale='log', )#ylim=(2e-1,2e3))
sns.despine()
for name, ax in g.axes_dict.items(): 
    ax.grid()
    ax.axhline(1, color='black', zorder=1)
g.figure.savefig(rd.outfile(output_path/(f'line_'+mir+'_controls_norm-to-base.svg')), bbox_inches='tight')

In [ ]:
plot_df = stats[~((stats['group']=='controller') & (stats['ts_kind']=='T'))]
plot_df.sort_values('group', key=lambda x: x.map(group_order), inplace=True)
g = sns.catplot(data=plot_df, x='group', y='output_gmean', hue='construct', palette=controls_palette, kind='swarm',
                legend=False, log_scale=True, size=8,)
g.figure.savefig(rd.outfile(output_path/(f'scatter_'+mir+'_controls.svg')), bbox_inches='tight')

In [ ]:
plot_df = stats[~((stats['group']=='controller') & (stats['ts_kind']=='T'))]
plot_df.sort_values('group', key=lambda x: x.map(group_order), inplace=True)
g = sns.catplot(data=plot_df, x='group', y='output_gmean', hue='construct', palette=controls_palette,
                legend=False, log_scale=True, kind='point', estimator='mean', errorbar='se', dodge=0.5, 
                markeredgecolor='white', markeredgewidth=1, markersize=12)
g.ax.axhline(plot_df.loc[plot_df['group']=='base', 'output_gmean'].mean(), color='black', zorder=0)
g.ax.set(xlabel='')
labels = stats.drop_duplicates('group').copy()
labels['microRNA'] = labels['miR_loc'].replace({'na': '–', 'CDS': '+', 'UTR': '+'})
labels['target site'] = labels['group'].replace({'base': '–', 'miR': '–', 'ts3': '3\'', 'ts5': '5\'', 'controller': 'NT'})
rd.plot.generate_xticklabels(labels, 'group', ['microRNA', 'target site'])
g.figure.savefig(rd.outfile(output_path/(f'stat_'+mir+'_controls.svg')), bbox_inches='tight')

In [ ]:
plot_df = stats_norm_base[(stats_norm_base['group']!='base') & ~((stats_norm_base['group']=='controller') & (stats_norm_base['ts_kind']=='T'))]
plot_df.sort_values('group', key=lambda x: x.map(group_order), inplace=True)
g = sns.catplot(data=plot_df, x='group', y='output_norm_to_base_gmean', hue='construct', palette=controls_palette,
                legend=False, log_scale=True, kind='point', estimator='mean', errorbar='se', dodge=True, 
                markeredgecolor='white', markeredgewidth=1, markersize=12)
g.ax.axhline(1, color='black', zorder=0)
g.set(xlabel='')
rd.plot.generate_xticklabels(labels, 'group', ['microRNA', 'target site'])
g.figure.savefig(rd.outfile(output_path/(f'stat_'+mir+'_controls_norm-to-base.svg')), bbox_inches='tight')

So it looks like 3'UTR miR and 5'UTR target sites overall reduce expression, while other placements do not. This suggests that any changes from baseline (no miR & no TS) seen in design 1 (CDS miR, 3'UTR target sites) are due to microRNA knockdown alone.

##### Now let's look at the targeting controller

In [ ]:
plot_df = mir_df
g = sns.displot(data=plot_df[plot_df['group']=='controller'], x='output', hue='construct', 
                palette=construct_palette, row='biorep', col='design', kind='kde', log_scale=True, legend=False, 
                common_norm=False, facet_kws=dict(margin_titles=True, sharex=True, sharey=True), )
for (biorep, design), ax in g.axes_dict.items():
    sns.kdeplot(data=plot_df[(plot_df['group']=='base') & (plot_df['biorep']==biorep)], x='output', 
                log_scale=True, legend=False, common_norm=False, ax=ax, color='black', ls=':')
g.figure.savefig(rd.outfile(output_path/(f'hist_'+mir+'_controller.svg')), bbox_inches='tight')

In [ ]:
plot_df = df_binned[(df_binned['group']=='controller')]
plot_df.sort_values('group', key=lambda x: x.map(group_order), inplace=True)
g = sns.relplot(data=plot_df, row='biorep', col='design', facet_kws=dict(sharex=True, sharey=True, margin_titles=True,), kind='line',
                    height=3, aspect=1.1, x='bin_marker_median', y='output', hue='construct',
                    palette=construct_palette, legend=False, dashes=False, style='construct', markers=construct_markers,
                    estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
g.set(xscale='log', yscale='log',)
sns.despine()
for name, ax in g.axes_dict.items(): ax.grid()
g.figure.savefig(rd.outfile(output_path/(f'line_'+mir+'_controller.svg')), bbox_inches='tight')

In [ ]:
plot_df = df_binned_combined[(df_binned_combined['group']=='controller')]
plot_df.sort_values('group', key=lambda x: x.map(group_order), inplace=True)
g = sns.relplot(data=plot_df, col='design', facet_kws=dict(sharex=True, sharey=True, margin_titles=True,), kind='line',
                    height=4, aspect=0.8, x='bin_marker_median', y='output', hue='construct',
                    palette=construct_palette, legend=False, dashes=False, style='construct', markers=construct_markers,
                    estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
g.set(xscale='log', yscale='log',)
sns.despine()
for name, ax in g.axes_dict.items(): ax.grid()
g.figure.savefig(rd.outfile(output_path/(f'line_'+mir+'_controller-combined-bioreps.svg')), bbox_inches='tight')

In [ ]:
# Normalize output to gmean of output for corresponding NT controller for each design
def normalize_output_to_NT(df):
    df = df.copy()
    normalizer = sp.stats.gmean(df.loc[(df['group']=='controller') & (df['ts_kind']=='NT'), 'output'])
    df['output_norm_to_NT'] = df['output'].astype(float) / normalizer
    return df

by = ['design','bin_marker_median','biorep','exp']
df_norm_NT = df_binned.groupby(by)[df_binned.columns].apply(normalize_output_to_NT).reset_index(drop=True).dropna()
display(df_norm_NT)

In [ ]:
# Group and compute stats
stat_list = [np.mean, np.std, sp.stats.gmean, sp.stats.gstd, sp.stats.variation]
grouped = df_norm_NT.groupby(by=['construct','biorep','exp'])
stats_norm_NT = grouped[['output_norm_to_NT']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_norm_NT.columns = stats_norm_NT.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_norm_NT['count'] = grouped['marker'].count().reset_index()['marker']
stats_norm_NT = stats_norm_NT.merge(metadata, how='left', on='construct')
stats_norm_NT.sort_values(['design','ts_kind','ts_num'], inplace=True)

display(stats_norm_NT)

In [ ]:
def normalize_stat_to_NT(df):
    df = df.copy()
    normalizer = df.loc[(df['group']=='controller') & (df['ts_kind']=='NT'), 'output']
    df['output_norm_to_NT'] = df['output'].astype(float) / normalizer
    return df

by = ['design','bin_marker_median','biorep','exp']
df_norm_NT = df_binned.groupby(by)[df_binned.columns].apply(normalize_output_to_NT).reset_index(drop=True).dropna()
display(df_norm_NT)

In [ ]:
plot_df = df_norm_NT[(df_norm_NT['group']=='controller')]
plot_df.sort_values('group', key=lambda x: x.map(group_order), inplace=True)
g = sns.relplot(data=plot_df, row='biorep', col='design', facet_kws=dict(sharex=True, sharey=True, margin_titles=True,), kind='line',
                    height=3, aspect=1.3, x='bin_marker_median', y='output_norm_to_NT', hue='construct',
                    palette=construct_palette, legend=False, dashes=False, style='construct', markers=construct_markers,
                    estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
g.set(xscale='log', yscale='log',)
sns.despine()
for name, ax in g.axes_dict.items(): 
    ax.grid()
    ax.axhline(1, color='black', zorder=1)
g.figure.savefig(rd.outfile(output_path/(f'line_'+mir+'_controller_norm-to-NT.svg')), bbox_inches='tight')

In [ ]:
plot_df = stats[(stats['group']=='controller')]
plot_df.sort_values(['design', 'ts_kind'], inplace=True)
stat_list = ['output_gmean', 'output_std', 'output_variation']
fig, axes = plt.subplots(1,3, figsize=(12,4), gridspec_kw=dict(wspace=0.5))
for i,stat in enumerate(stat_list):
    ax = axes[i]
    sns.scatterplot(data=plot_df, x='construct', y=stat, hue='construct', palette=construct_palette,
                legend=False, s=100, ax=ax, style='construct', markers=construct_markers)
    ax.set(xlim=(-0.5,5.5))
    if stat != 'output_variation': ax.set(yscale='log')
    else: ax.set(ylim=(0,ax.get_ylim()[1]))
    rd.plot.generate_xticklabels(plot_df.drop_duplicates('construct'), 'construct', ['design', 'ts_kind'], ax=ax)
    sns.despine(ax=ax)

fig.savefig(rd.outfile(output_path/(f'stat_'+mir+'_controller.svg')), bbox_inches='tight')

In [ ]:
plot_df = stats[(stats['group']=='controller')]
plot_df.sort_values(['design','ts_kind'], inplace=True)
stat_list = ['output_gmean', 'output_std', 'output_variation']
fig, axes = plt.subplots(1,3, figsize=(12,4), gridspec_kw=dict(wspace=0.5))
for i,stat in enumerate(stat_list):
    ax = axes[i]
    sns.pointplot(data=plot_df, x='construct', y=stat, hue='construct', palette=construct_palette,
                legend=False, estimator='mean', errorbar='se', dodge=0.5, 
                markeredgecolor='white', markeredgewidth=1, markersize=12, ax=ax)
    ax.set(xlabel='', xlim=(-0.5,5.5))
    rd.plot.generate_xticklabels(plot_df.drop_duplicates('construct'), 'construct', ['design', 'ts_kind'], ax=ax)
    sns.despine(ax=ax)
    if stat != 'output_variation': ax.set(yscale='log')
    else: ax.set(ylim=(0,ax.get_ylim()[1]))
fig.savefig(rd.outfile(output_path/(f'stat_'+mir+'_controller_summary.svg')), bbox_inches='tight')